In [1]:
import os
from sklearn.metrics import f1_score, recall_score, precision_score
import sys
import numpy as np
import torch
import matplotlib.pyplot as plt
import pandas as pd

sys.path.append('../src')
import BP

In [2]:
'''UTILS'''

def decode_TSP(path):
    
    '''
    return configuration from path
    
    path:: a list of numbers (string format), representing an optimal TSP path
    '''
    
    num_nodes = int(len(path)-1)
    
    edges = [(int(i)-1,int(j)-1) for i,j in zip(path,path[1:])]
    
    n = np.zeros((num_nodes,num_nodes))
    
    for edge in edges:
        n[edge] = 1
        
    return n


def cost_of_TSP(graph,path):
    '''
    returns cost of TSP path
    '''
    tsp = torch.tensor(decode_TSP(path))
    
    w = BP.cost_matrix(graph)
    
    cost = torch.sum(torch.masked_select(w,tsp==1),dtype=float)
    return np.int(cost)


def read_concorde_data(path,n_samples=None):
    
    data = open(path, "r").readlines()
    
    if not n_samples: n_samples = len(data)
    
    graphs = [data[i][:data[i].index('output')].split() for i in range(n_samples)]
    paths = [data[i][data[i].index('output')+6:].split() for i in range(n_samples)]
    costs = [cost_of_TSP(g,p) for g,p in zip(graphs,paths)]
    tsp_paths = [decode_TSP(i) for i in paths]
    
    assert np.all([len(paths[i]) - 1 == len(graphs[i])/2 for i in range(n_samples)]) == 1
    
    return graphs, tsp_paths, costs

def compute_metrics(true,pred):
    
    '''
    return average precision, recall and f1
    
    true:: list of array-like objects, true labels
    pred:: list of array-like objects, "predicted" labels (from weighted 2-matching)
    '''
    
    metrics = []
    for i,j in zip(true,pred):
        metrics.append([precision_score(i,j),recall_score(i,j),f1_score(i,j)])
    metrics = np.array(metrics)
    
    return np.around(np.mean(metrics,axis=0),decimals=2)

In [3]:
data_dir = '../tsp30-50'
f_name = 'tsp30-50_train'
file = f'{f_name}.txt'
path = os.path.join(data_dir,file)

graphs,tsp_paths,costs = read_concorde_data(path)

In [4]:
print(len(graphs))

10000


In [5]:
#sanity check
matchings = f'../out/BPmatch/tsp30-50_train_match.pt'
t = torch.load(matchings)
assert len(t) == len(graphs)

## DIAGNOSTICS

In [6]:
match_dir = '../out'

keys = ['BASE','DAMP','BAYATI','BAYATI-4']
ids = ['','_damp','_bayati','_bayati_4']
vals = [os.path.join(match_dir,f'BPmatch{i}','tsp30-50_train_match.txt') for i in ids]

experiments = {k:v for k,v in zip(keys,vals)}

experiments

{'BASE': '../out/BPmatch/tsp30-50_train_match.txt',
 'DAMP': '../out/BPmatch_damp/tsp30-50_train_match.txt',
 'BAYATI': '../out/BPmatch_bayati/tsp30-50_train_match.txt',
 'BAYATI-4': '../out/BPmatch_bayati_4/tsp30-50_train_match.txt'}

In [7]:
for k,v in experiments.items():
    print(k,':', open(v, "r").readlines()[0]) 

BASE : max_iter=1000, thresh=10, d=0, b=2, rand_init=0, seed=0

DAMP : max_iter=1000, thresh=10, d=0.5, b=2, rand_init=0, seed=0

BAYATI : max_iter=1000, thresh=10, d=0, b=2, rand_init=0, seed=0

BAYATI-4 : max_iter=1000, thresh=10, d=0, b=4, rand_init=0, seed=0



In [8]:
dfs = []
for k,v, in experiments.items():
    temp = pd.read_csv(v, sep=" ",skiprows=1)
    temp['ID'] = k
    dfs.append(temp)
    temp['gap'] = (temp.cost_of_matching - costs) / costs
    temp['nodes'] = [len(i) for i in tsp_paths]
    temp['rel_violations'] = temp.n_violations / (temp.nodes*2)
df = pd.concat(dfs)
df.head()

,cost_of_matching,n_violations,converged,ID,gap,nodes,rel_violations
0,0,78,False,BASE,-1.00,45,0.866667
1,1,36,False,BASE,-0.80,32,0.562500
2,2,82,False,BASE,-0.60,49,0.836735
3,1,52,False,BASE,-0.75,36,0.722222
4,1,56,False,BASE,-0.80,38,0.736842


In [9]:
print(df['ID'].unique())
print(len(df))

['BASE' 'DAMP' 'BAYATI' 'BAYATI-4']
40000


In [10]:
df.groupby('ID').agg({'rel_violations': 'mean',
                         'converged' : lambda x: x.sum() / 10000,
                    'gap': 'mean'})

,rel_violations,converged,gap
ID,,,
BASE,0.782357,0.1534,-0.386418
BAYATI,0.066805,0.1647,1.059747
BAYATI-4,1.000000,0.0439,5.828032
DAMP,0.075188,0.3695,0.888828


## TSP OVERLAP

In [218]:
matchings = [os.path.join(match_dir,f'BPmatch{i}','tsp30-50_train_match.pt') for i in ids]

['BASE', 'DAMP', 'BAYATI', 'BAYATI-4']

In [209]:
bp_vals = [torch.load(i) for i in matchings]
bp_vals = [[t[i].flatten() for i in range(len(t))] for t in bp_vals]

In [28]:
tsp_paths = [i.ravel() for i in tsp_paths] #temporary patch, not elegant

In [232]:
bp_dict = {k:v for k,v in zip(keys,bp_vals)}

In [226]:
metrics_dict = {}
for i,j in zip(keys,bp_dict.values()):
    metrics_dict[i] = compute_metrics(tsp_paths,j)

In [227]:
metrics_dict #precision (on 1), recall (on 1), f1_score 

{'BASE': array([0.16, 0.31, 0.2 ]),
 'DAMP': array([0.38, 0.73, 0.5 ]),
 'BAYATI': array([0.36, 0.75, 0.49]),
 'BAYATI-4': array([0.2 , 0.97, 0.33])}

In [20]:
final_bp = os.path.join(match_dir,'BPmatch_damp_bayati','tsp30-50_train_match.pt')

In [21]:
final_bp = torch.load(final_bp)

In [23]:
final_bp = [i.flatten() for i in final_bp]

In [30]:
compute_metrics(tsp_paths,final_bp)

array([0.37, 0.75, 0.49])

In [32]:
final_txt = os.path.join(match_dir,'BPmatch_damp_bayati','tsp30-50_train_match.txt')

In [33]:
df = pd.read_csv(final_txt, sep=" ",skiprows=1)
df['gap'] = (df.cost_of_matching - costs) / costs
df['nodes'] = [len(i) for i in tsp_paths]
df['rel_violations'] = df.n_violations / (df.nodes*2)

In [34]:
df.head()

,cost_of_matching,n_violations,converged,gap,nodes,rel_violations
0,11,12,False,1.2,2025,0.002963
1,9,4,True,0.8,1024,0.001953
2,10,8,False,1.0,2401,0.001666
3,8,4,False,1.0,1296,0.001543
4,10,4,False,1.0,1444,0.001385


In [37]:
df[['gap','rel_violations']].mean()

gap               1.041668
rel_violations    0.001467
dtype: float64

In [38]:
df['converged'].value_counts() / 10000

False    0.6231
True     0.3769
Name: converged, dtype: float64